In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json

import utilities.functions as functions
from utilities.functions import (
    retidos,
    calcula_viabilidade,
    analisar_retencao,
)

In [ ]:
import os
import os
os.getcwd()
os.listdir("dados")
import os
os.listdir("dados/gold")


In [ ]:
publico_janeiro_dezembro = pd.read_parquet("dados/gold/publico_janeiro_dezembro.parquet")



In [ ]:
publico_janeiro_dezembro.head(10)

Calculando retencao considerando dois ou + pedidos

In [ ]:
df_1,clientes_retidos_1=retidos(publico_janeiro_dezembro, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)

In [ ]:
retencao_pedido_1=analisar_retencao(df_1)
retencao_pedido_1

Calculando retencao considerando tres ou + pedidos

In [ ]:
df_2,clientes_retidos_2=retidos(publico_janeiro_dezembro, mes0=12, mes1=1,pedidos=2)
print(clientes_retidos_2)

In [ ]:
retencao_pedido_2=analisar_retencao(df_2)
retencao_pedido_2

Viabilidade 

In [ ]:
resultados, agg = calcula_viabilidade(
    publico_janeiro_dezembro,
    mes_campanha=12,
    mes_seguinte=1,
    coupon_value=10.0,   
    margin_rate=0.12     
)

resultados


Calculando retencao separadamente para outliers

In [ ]:
orderns_de=publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==12]

In [ ]:
distance = 1.5 * (np.nanpercentile(orderns_de['total_amount_mes'], 75) - np.nanpercentile(orderns_de['total_amount_mes'], 25))
lim_sup=distance + np.nanpercentile(orderns_de['total_amount_mes'], 75)

In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
orderns_de['total_amount_mes'].plot(kind='box', xlim=(0, 500), vert=False, flierprops=red_square, figsize=(16,2))

In [ ]:
orderns_de['total_amount_mes'].describe().round(2)

In [ ]:
pb_drop_dj = publico_janeiro_dezembro.drop(
    publico_janeiro_dezembro[
        (publico_janeiro_dezembro['total_amount_mes'] == 0) |
        (publico_janeiro_dezembro['total_amount_mes'] > lim_sup)
    ].index,
    axis=0
)

In [ ]:
pb_drop_dj

In [ ]:
plt.figure(figsize=(12, 6))

sns.boxplot(
    data=pb_drop_dj,
    x='order_created_month',               
    y='total_amount_mes',   
    hue='is_target',           
    showfliers=False          
)

plt.title('Distribuição do valor gasto por mês e target')
plt.xlabel('Mês')
plt.ylabel('Total Gasto no Mês')
plt.legend(title='Grupo')
plt.tight_layout()
plt.show()

In [ ]:
pb_drop_dj_1,clientes_retidos_1=retidos(pb_drop_dj, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)

In [ ]:
pb_drop_dj_s=analisar_retencao(pb_drop_dj_1)
pb_drop_dj_s

In [ ]:
resultados_out, agg_out = calcula_viabilidade(
    pb_drop_dj,
    mes_campanha=12,
    mes_seguinte=1,
    coupon_value=10.0,   
    margin_rate=0.12     
)

resultados_out
